## Comparisons

This code can execute the following three variants of losses:  

1. **Variant 1:**  Purely Physics  
   $L_{\theta} = L_{\text{PDE}}$  
   Use $n_{\text{used}} = 0$  

2. **Variant 2:**  Physics + Data  
   $L_{\theta} = L_{\text{PDE}} + \Sigma_{i=1}^{n_{\text{used}}}\| u_i - \hat{u}_i \|_2^2$  
   Use $n_{\text{used}} \in (0, 1000]$  

All the results presented were obtained as follows:
1. By estimating the gradients in the physics-informed loss terms using forward mode automatic differentiation (AD).
2. The output field values at given grid points were computed in one forward pass of the network using the einsum function.

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torchsummary import summary
import torch.distributions as td
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import argparse
import random
import os
import time 
from scipy.io import loadmat

import sys
sys.path.append("../..")
from utils.networks import *
from utils.deeponet_networks_1d import *
from utils.visualizer_misc import *
from utils.visualizer_1d import *
from utils.forward_autodiff import *
from utils.misc import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Tag this cell with 'parameters'
# parameters
seed = 0 # Seed number.
n_used = 0 #100 # Number of full training fields used for estimating the data-driven loss term
save = True # Save results.

In [ ]:
if save == True:
    resultdir = os.path.join(os.getcwd(),'results','a_Vanilla-NO','seed='+str(seed)+'_n_used='+str(n_used)) 
    if not os.path.exists(resultdir):
        os.makedirs(resultdir)
else:
    resultdir = None

In [ ]:
set_seed(seed)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Load the data
data = loadmat(os.path.join('..','..','data/1D_Burgers_transport_dynamics_t=0to1/Burger.mat')) # Load the .mat file
#print(data)
print(data['tspan'].shape)
print(data['input'].shape)  # Random Initial conditions: Gaussian random fields, Nsamples x 101, each IC sample is (1 x 101)
print(data['output'].shape) # Time evolution of the solution field: Nsamples x 101 x 101.
                             # Each field is 101 x 101, rows correspond to time and columns respond to location.
                             # First row corresponds to solution at t=0 (1st time step)
                             # and next  row corresponds to solution at t=0.01 (2nd time step) and so on.
                             # last row correspond to solution at t=1 (101th time step).

In [ ]:
# Convert NumPy arrays to PyTorch tensors
inputs = torch.from_numpy(data['input'][0:1500]).float().to(device)
outputs = torch.from_numpy(data['output'][0:1500]).float().to(device)

t_span = torch.from_numpy(data['tspan'].flatten()).float().to(device)
x_span = torch.linspace(0, 1, data['output'].shape[2]).float().to(device)
nt, nx = len(t_span), len(x_span) # number of discretizations in time and location.
print("nt =",nt, ", nx =",nx)
print("Shape of t-span and x-span:",t_span.shape, x_span.shape)
print("t-span:", t_span)
print("x-span:", x_span)

# Estimating grid points
T, X = torch.meshgrid(t_span, x_span)
# print(T)
# print(X)
grid = torch.vstack((T.flatten(), X.flatten())).T
print("Shape of grid:", grid.shape) # (nt*nx, 2)
print("grid:", grid) # time, location

# Split the data into training and testing samples
inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(inputs, outputs, test_size=500, random_state=seed)

# Check the shapes of the subsets
print("Shape of inputs_train:", inputs_train.shape)
print("Shape of inputs_test:", inputs_test.shape)
print("Shape of outputs_train:", outputs_train.shape)
print("Shape of outputs_test:", outputs_test.shape)
print('#'*100)

In [ ]:
# Of these full training fields available I am using only n_used fields for estimating the data-driven loss term in the PI-Latent-NO
inputs_train_used = inputs_train[:n_used, :]
print("Shape of inputs_train_used:", inputs_train_used.shape)
outputs_train_used = outputs_train[:n_used, :, :]
print("Shape of outputs_train_used:", outputs_train_used.shape)

In [ ]:
"""
input_neurons_branch: Number of input neurons in the branch net.
input_neurons_trunk: Number of input neurons in the trunk net.
p: Number of output neurons in both the branch and trunk net.
"""
p = 128 # Number of output neurons in both the branch and trunk net.

input_neurons_branch = nx # m
branch_net = DenseNet(layersizes=[input_neurons_branch] + [64]*3 + [p], activation=nn.SiLU()) #nn.LeakyReLU() #nn.Tanh()
branch_net.to(device)
# print(branch_net)
print('BRANCH-NET SUMMARY:')
summary(branch_net, input_size=(input_neurons_branch,))  
print('#'*100)

input_neurons_trunk = 2 # 2 corresponds to t and x
trunk_net = DenseNet(layersizes=[input_neurons_trunk] + [64]*3 + [p], activation=nn.SiLU()) #nn.LeakyReLU() #nn.Tanh()
trunk_net.to(device)
# print(trunk_net)
print('TRUNK-NET SUMMARY:')
summary(trunk_net, input_size=(input_neurons_trunk,))
print('#'*100)

model = Vanilla_NO_model(branch_net, trunk_net)
model.to(device);

In [ ]:
num_learnable_parameters = count_learnable_parameters(branch_net) + count_learnable_parameters(trunk_net)
print("Total number of learnable parameters:", num_learnable_parameters)

In [ ]:
def loss_pde_residual(net, initial_fields, t, x):
    
    u = net(initial_fields, torch.hstack([t, x])) # u is (bs, neval_c)
    
    # Using forward automatic differention to estimate derivatives in the physics informed loss
    tangent_t, tangent_x = torch.ones(t.shape).to(device), torch.ones(x.shape).to(device)
    ut  = FWDAD_first_order_derivative(lambda t: net(initial_fields, torch.hstack([t, x])), t, tangent_t) # (bs, neval_c)
    ux  = FWDAD_first_order_derivative(lambda x: net(initial_fields, torch.hstack([t, x])), x, tangent_x) # (bs, neval_c)
    uxx = FWDAD_second_order_derivative(lambda x: net(initial_fields, torch.hstack([t, x])), x, tangent_x) # (bs, neval_c)
    
    pde_residual = (ut + (u*ux) - (0.01*uxx))**2
    
    return torch.mean(pde_residual)

In [ ]:
def loss_pde_bcs(net, initial_fields, t, x):
    
    t_b1, x_b1 = t[0], x[0]
    t_b2, x_b2 = t[1], x[1]

    u_b1 = net(initial_fields, torch.hstack([t_b1, x_b1])) # u is (bs, neval_b)
    u_b2 = net(initial_fields, torch.hstack([t_b2, x_b2])) # u is (bs, neval_b)
    
    # Using forward automatic differention to estimate derivatives in the physics informed loss
    tangent_x_b1, tangent_x_b2 = torch.ones(x_b1.shape).to(device), torch.ones(x_b2.shape).to(device)
    ux_b1 = FWDAD_first_order_derivative(lambda x_b1: net(initial_fields, torch.hstack([t_b1, x_b1])), x_b1, tangent_x_b1) # (bs, neval_b)
    ux_b2 = FWDAD_first_order_derivative(lambda x_b2: net(initial_fields, torch.hstack([t_b2, x_b2])), x_b2, tangent_x_b2) # (bs, neval_b)

    pde_bc1 = (u_b1 - u_b2)**2
    pde_bc2 = (ux_b1 - ux_b2)**2
    
    return torch.mean(pde_bc1) + torch.mean(pde_bc2)

In [ ]:
def loss_pde_ic(net, initial_fields, t, x):
    
    u_ic = net(initial_fields, torch.hstack([t, x])) # u is (bs, neval_i)
    
    bs_ = initial_fields.shape[0]
    ic_values_ = torch.zeros((bs_, x.shape[0], 1)).to(device)
    for j in range(bs_):
        ic_values_[j] = linear_interpolation_1D(x, x_span, initial_fields[j]) # initial condition: u_0(x) values
    ic_values = ic_values_.reshape(-1, x.shape[0]) # (bs, neval_i)
    
    pde_ic = (u_ic - ic_values)**2
    
    return torch.mean(pde_ic)

In [ ]:
def collocation_points(tc_span, xc_span, neval_dict):
    tc = tc_span.repeat_interleave(neval_dict['loc']).unsqueeze(-1)
    xc = xc_span.flatten().repeat(neval_dict['t']).unsqueeze(-1)
    return tc, xc

In [ ]:
start_time = time.time()

bs = 64 # Batch size
neval_t = 512 # Number of time points at which output field is evaluated.
neval_x = 512
# neval_loc = neval_x  # Number of locations at which output field is evaluated at each time point.

neval_c = {'t': neval_t, 'loc': neval_x}  # Number of collocation points within the domain.
neval_b = {'t': neval_t, 'loc': 1}        # Number of collocation points on each boundary.
neval_i = {'t': 1, 'loc': neval_x}        # Number of collocation points at t=0.
        
# Training
optimizer = torch.optim.Adam(model.parameters(), lr=1.5*1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=25000, gamma=0.1) # gamma=0.8

iteration_list, loss_list, learningrates_list = [], [], []
datadriven_loss_list, pinn_loss_list = [], []
test_iteration_list, test_loss_list = [], []

n_iterations = 50000
for iteration in range(n_iterations):
    
    if n_used == 0:
        datadriven_loss = torch.tensor([0.]).to(device)
        # print('*********')
    else:
        indices_datadriven = torch.randperm(n_used).to(device) # Generate random permutation of indices
        inputs_train_used_batch = inputs_train_used[indices_datadriven[0:bs]]
        outputs_train_used_batch = outputs_train_used.reshape(-1, nt*nx)[indices_datadriven[0:bs]]
        # print(f"Shape of inputs_train_used_batch:", inputs_train_used_batch.shape) # (bs, nx)
        # print(f"Shape of outputs_train_used_batch:", outputs_train_used_batch.shape) # (bs, nt*nx)

        predicted_values = model(inputs_train_used_batch, grid)  # (bs, neval) = (bs, nt*nx)
        target_values = outputs_train_used_batch # (bs, nt*nx)
        datadriven_loss = nn.MSELoss()(predicted_values, target_values)
        # print('*********')
    
    # Shuffle the train data using the generated indices
    num_samples = len(inputs_train)
    indices = torch.randperm(num_samples).to(device) # Generate random permutation of indices
    inputs_batch = inputs_train[indices[0:bs]]
    #print(f"Shape of inputs_train_batch:", inputs_batch.shape) # (bs, nx)
        
    # points within the domain
    tc_span = td.uniform.Uniform(0., 1.).sample((neval_c['t'], 1)).to(device)
    xc_span = td.uniform.Uniform(0., 1.).sample((neval_c['loc'], 1)).to(device)
    tc, xc= collocation_points(tc_span, xc_span, neval_c)

    # boundary points on the 2 boundaries (hard-coded)
    tb = [td.uniform.Uniform(0., 1.).sample((neval_b['t'], 1)).to(device),
          td.uniform.Uniform(0., 1.).sample((neval_b['t'], 1)).to(device)]

    xb = [torch.full((neval_b['t'], 1), 0.).to(device),
          torch.full((neval_b['t'], 1), 1.).to(device)]

    # initial points
    ti = torch.full((neval_i['loc'], 1), 0.).to(device)
    xi = td.uniform.Uniform(0., 1.).sample((neval_i['loc'], 1)).to(device)
    
    pinn_loss = (loss_pde_residual(model, inputs_batch, tc, xc) 
           + loss_pde_bcs(model, inputs_batch, tb, xb) 
           + loss_pde_ic(model, inputs_batch, ti, xi))

    optimizer.zero_grad()
    loss = datadriven_loss + pinn_loss
    loss.backward()
    # torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=1.0)
    optimizer.step()
    scheduler.step()

    if iteration % 500 == 0:
        # Test loss calculation
        model.eval()  # Set model to evaluation mode
        with torch.no_grad():
            test_predicted_values = model(inputs_test, grid)  # (bs, neval) = (bs, nt*nx)
            test_loss = nn.MSELoss()(test_predicted_values, outputs_test.reshape(-1, nt*nx))
            test_iteration_list.append(iteration)
            test_loss_list.append(test_loss.item())  
        model.train()  # Switch back to training mode
        print('Iteration %s -' % iteration, 'loss = %f,' % loss,
              'data-driven loss = %f,' % datadriven_loss,'pinn loss = %f,' % pinn_loss,
              'learning rate = %f,' % optimizer.state_dict()['param_groups'][0]['lr'],
              'test loss = %f' % test_loss)

    iteration_list.append(iteration)
    loss_list.append(loss.item())
    datadriven_loss_list.append(datadriven_loss.item())
    pinn_loss_list.append(pinn_loss.item())
    learningrates_list.append(optimizer.state_dict()['param_groups'][0]['lr'])


if save == True:
    np.save(os.path.join(resultdir,'iteration_list.npy'), np.asarray(iteration_list))
    np.save(os.path.join(resultdir,'loss_list.npy'), np.asarray(loss_list))
    np.save(os.path.join(resultdir, 'datadriven_loss_list.npy'), np.asarray(datadriven_loss_list))
    np.save(os.path.join(resultdir, 'pinn_loss_list.npy'), np.asarray(pinn_loss_list))
    np.save(os.path.join(resultdir,'learningrates_list.npy'), np.asarray(learningrates_list))
    np.save(os.path.join(resultdir,'test_iteration_list.npy'), np.asarray(test_iteration_list))
    np.save(os.path.join(resultdir, 'test_loss_list.npy'), np.asarray(test_loss_list)) 

plot_loss_terms(resultdir, iteration_list, loss_list, datadriven_loss_list, pinn_loss_list, save)  

plot_training_loss(resultdir, iteration_list, loss_list, save)

plot_testing_loss(resultdir, test_iteration_list, test_loss_list, save)

plot_training_testing_loss(resultdir, iteration_list, loss_list, test_iteration_list, test_loss_list, save)

plot_learningrates(resultdir, iteration_list, learningrates_list, save)
    
# end timer
end_time = time.time()
training_time = end_time - start_time # in sec
 
runtime_per_iter = training_time/n_iterations # in sec/iter

In [ ]:
if save == True:
    torch.save(model.state_dict(), os.path.join(resultdir,'model_state_dict.pt'))
# model.load_state_dict(torch.load(os.path.join(resultdir,'model_state_dict.pt'), map_location=device))

In [ ]:
# Predictions
branch_inputs = inputs_test # (bs, m) = (bs, nx) 
trunk_inputs = grid # (neval, 2) = (nt*nx, 2)
predictions_test = model(branch_inputs, trunk_inputs) # (bs, neval) = (bs, nt*nx)

mse_list, r2score_list, relerror_list = [], [], []

for i in range(inputs_test.shape[0]):

    prediction_i = predictions_test[i].reshape(1, -1) # (1, nt*nx)
    target_i = outputs_test[i].reshape(1, -1) # (1, nt*nx)
    
    mse_i = F.mse_loss(prediction_i.cpu(), target_i.cpu())
    r2score_i = metrics.r2_score(target_i.flatten().cpu().detach().numpy(), prediction_i.flatten().cpu().detach().numpy()) 
    relerror_i = np.linalg.norm(target_i.flatten().cpu().detach().numpy() - prediction_i.flatten().cpu().detach().numpy()) / np.linalg.norm(target_i.flatten().cpu().detach().numpy())
    
    mse_list.append(mse_i.item())
    r2score_list.append(r2score_i.item())
    relerror_list.append(relerror_i.item())
    
    if (i+1) % 10 == 0:
        plot_predictions(i, resultdir, target_i, prediction_i, x_span, inputs_test, X, T, nt, nx, r'$u_0(x)$', 'Initial field', 'jet', save)

mse = sum(mse_list) / len(mse_list)
print("Mean Squared Error Test:\n", mse)
r2score = sum(r2score_list) / len(r2score_list)
print("R2 score Test:\n", r2score)
relerror = sum(relerror_list) / len(relerror_list)
print("Rel. L2 Error Test:\n", relerror)

In [ ]:
test_dict = {
    "inputs_test": inputs_test.cpu(),
    "outputs_test": outputs_test.cpu(),
    "predictions_test": predictions_test.reshape(-1, nt, nx).cpu()
}
for key, value in test_dict.items():
    print(f"Shape of {key}: {value.shape}")
print(colored('#'*230, 'green'))

if save == True:
    torch.save(test_dict, os.path.join(resultdir,'test_dict.pth'))

In [ ]:
performance_metrics(mse, r2score, relerror, training_time, runtime_per_iter, resultdir, save)